# Class-E amplifier

* Switching amplifier
* High efficiency
* Single switching element

# Calculation
A lot of the design work has been done in the [Zombocalyps-project](https://github.com/LieBtrau/zombocalypse-emcomms/blob/main/amplifier/amplifier.ipynb).

# Part selection
## Transistor
### Transistor criteria
* breakdown voltage (Vds) > 7x supply voltage
* Switching energy = gate charge * gate voltage
* ON resistance

### Transistor parameters
So what's more important : total gate charge, Ciss, turn-on delay, rise time, fall time, turn-off delay?

* Rise time and fall time are 0s, because it's a class-E amplifier, Vds is always 0V at the transition
* Transistor turn-on speed is determined by : driver strength (should have low output impedance) and total gate charge of the transistor.
* Energy required for switching is the product of the gate charge and the gate voltage. 
* It doesn't make much sense to drive the gate higher than the miller plateau.
* [Total gate charge and Vgs_th are more important than Ciss as selection criteria](https://www.infineon.com/dgdl/Infineon-Use_Gate_Charge_to_Design_the_Gate_Drive_Circuit_for_Power_MOSFETs_and_IGBTs-AN-v01_00-EN.pdf).

### Reference
[AN608A Power MOSFET Basics: Understanding Gate Charge and Using it to Assess Switching Performance](https://www.vishay.com/docs/73217/an608a.pdf)

### Selection
Digikey engine seems to be broken, so I'll use Mouser instead:
* Mounting style: SMD
* Number of channels: 1
* Transistor polarity: N-channel
* Vds breakdown : >= 100V (> 7x supply voltage)
* Rds_on : <= 0.3 Ohm
* Vgs_th : <= 2V
* Qg gate charge : <= 5nC

Sort the results by Vgs_th.  Then multiply the gate charge by the gate voltage to get the switching energy.  The lower the better.
1. **IRLML0100TRPBF**
2. SI3474DV-T1-BE3
3. SSM3K361

In [22]:
import math
Vgs = 5 # Gate-source voltage [V]

# IRLML0100
Rg = 1.3 # Internal gate resistance [Ohm], add output resistance of the driver here
Ciss = 290e-12 # Input capacitance [F], at Vds=0V (see datasheet, fig.5)
Vth = 2.5 # Threshold voltage [V]
Vgp = 2.8 # Miller plateau voltage [V] (it's the value of the horizontal line in the gate charge curves in the datasheet, fig.6)

# Turn-on delay
td_on = Rg * Ciss * math.log(Vgs/(Vgs-Vgp)) # t2, eq. 12 = t1 + t_ir = eq. 11 + eq. 17
print(f"Turn-on delay: {td_on*1e12:.2f} ps")

# Turn-off delay
td_off = Rg * Ciss * math.log(Vgs/Vgp) # t4, eq. 14
print(f"Turn-off delay: {td_off*1e12:.2f} ps")

# Turn-off delay (alternative method)
td_off = Rg * Ciss * math.log(Vgs/(Vth)) # t4 + t6, eq. 14 + eq. 16
print(f"Turn-off delay (alternative method): {td_off*1e12:.2f} ps")


Turn-on delay: 309.51 ps
Turn-off delay: 218.59 ps
Turn-off delay (alternative method): 261.32 ps




## Inductor criteria
Use smallest inductor value possible to minimize core loss.
* DC resistance
* AC resistance (skin effect, proximity effect)
* core loss (frequency dependent)

# References
* [FesZ Electronics : Class E RF amplifier - Basics (1/3)](https://www.youtube.com/watch?v=Tgrakttus3c)